In [ ]:
# !/usr/bin/env python3
#coding=utf-8
import time
import binascii
import socket
import struct
import sys
from Arm_Lib import Arm_Device
import select

import cv2
import ipywidgets.widgets as widgets
import threading
import time
import enum

# Fix Address already in use
import os
os.system('fuser -kn tcp 65432 | awk')

always_read_camera = False

def bgr8_to_jpeg(value, quality=75):
    if value is None:
        print(value)
        return None
    return bytes(cv2.imencode('.jpg', value)[1])
   

width=512
height=512
image_widget = widgets.Image(format='jpeg', width=512, height=512)  #Set up camera display components
display(image_widget)                                               #Display camera assembly

image = cv2.VideoCapture(0)                           #Open camera


# cap.set(cv2.CAP_PROP_FRAME_WIDTH,width)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT,height)

# image.set(3,600)       
# image.set(4,500)
# image.set(5, 30)  #Set frame rate
# image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 40) #set brightness -64 - 64  0.0
# image.set(cv2.CAP_PROP_CONTRAST, 50)   #set contrast -64 - 64  2.0
# image.set(cv2.CAP_PROP_EXPOSURE, 156)  #set exposure 1.0 - 5000  156.0

ret, frame = image.read()     #Read camera data
converted_image = bgr8_to_jpeg(frame)
if always_read_camera or converted_image:
    image_widget.value = converted_image

HOST = ''  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)

# Get DOFBOT object
Arm = Arm_Device()
time.sleep(.1)
base_id = 1
shoulder_pan_id = 2
shoulder_lift_id = 3
elbow_id = 4
wrist_id = 5  
finger_id = 6
sleep_time_inbetween = 0.1

def receiveTextViaSocket(sock):
    unpacker = struct.Struct('f f f f f f')

    connection, client_address = sock.accept()
    data = connection.recv(unpacker.size)
    stringifiedData = str(binascii.hexlify(data))
    print("received" + stringifiedData)

    ACK_TEXT = 'text_not_received'
    if stringifiedData != "b''":
        unpacked_data = unpacker.unpack(data)
        print("unpacked:", str(unpacked_data))
        Arm.Arm_serial_servo_write(base_id, unpacked_data[base_id - 1], 500)
        time.sleep(sleep_time_inbetween)
        Arm.Arm_serial_servo_write(shoulder_pan_id, unpacked_data[shoulder_pan_id - 1], 300)
        time.sleep(sleep_time_inbetween)
        Arm.Arm_serial_servo_write(shoulder_lift_id, unpacked_data[shoulder_lift_id - 1], 300)
        time.sleep(sleep_time_inbetween)
        Arm.Arm_serial_servo_write(elbow_id, unpacked_data[elbow_id - 1], 300)
        time.sleep(sleep_time_inbetween)
        Arm.Arm_serial_servo_write(wrist_id, unpacked_data[wrist_id - 1], 300)
        time.sleep(sleep_time_inbetween)
        Arm.Arm_serial_servo_write(finger_id, unpacked_data[finger_id - 1], 300)
        time.sleep(sleep_time_inbetween)
        ACK_TEXT = 'text_received'

    encodedAckText = bytes('complete', 'utf-8')
    # send the encoded acknowledgement text
    connection.sendall(encodedAckText)
    connection.close()

def new_server():
    # Create a TCP/IP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = (HOST, PORT)
    sock.bind(server_address)
    sock.listen(1)

    unpacker = struct.Struct('f f f f f f')
    socks = [sock]
    while True:
        print('waiting for a connection')
        
        readySocks, _, _ = select.select(socks, [], [], 5)
        for sock in readySocks:
            receiveTextViaSocket(sock)
        
        ret, frame = image.read()
        converted_image = bgr8_to_jpeg(frame)
        if always_read_camera or converted_image:
            image_widget.value = converted_image
        time.sleep(sleep_time_inbetween)
               
 

new_server()

Image(value=b'', format='jpeg', height='512', width='512')

waiting for a connection
waiting for a connection
waiting for a connection
waiting for a connection
waiting for a connection
waiting for a connection
waiting for a connection
receivedb'0000b4420000b4420000b4420000b4420000b44200008c42'
unpacked: (90.0, 90.0, 90.0, 90.0, 90.0, 70.0)
waiting for a connection
receivedb'0000b4420000a0410000b4420000b4420000b44200008c42'
unpacked: (90.0, 20.0, 90.0, 90.0, 90.0, 70.0)
waiting for a connection
receivedb'0000b4420000a041000034420000b4420000b44200008c42'
unpacked: (90.0, 20.0, 45.0, 90.0, 90.0, 70.0)
waiting for a connection
receivedb'0000b4420000a041000034420000b4420000b44200001643'
unpacked: (90.0, 20.0, 45.0, 90.0, 90.0, 150.0)
waiting for a connection
receivedb'0000b44200004842000034420000b4420000b44200001643'
unpacked: (90.0, 50.0, 45.0, 90.0, 90.0, 150.0)
waiting for a connection
receivedb'0000b44200004842000034420000b4420000b44200001643'
unpacked: (90.0, 50.0, 45.0, 90.0, 90.0, 150.0)
waiting for a connection
receivedb'0000b442000048420000

In [ ]:
del Arm  # Release DOFBOT object